<a href="https://colab.research.google.com/github/Avani191/RPPOOP-Project/blob/main/proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
!pip install twilio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.7 MB/s eta 0:00:00


In [4]:
from twilio.rest import Client

account_sid = "ACae966fc28e59c51d1e297aa07ae5b68f"
auth_token = "d8feddac055c6917693cdc896b212f2d"

client = Client(account_sid, auth_token)

In [5]:
import numpy as np
import requests
from tensorflow.keras.models import Sequential
#from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
import nltk
nltk.download('vader_lexicon')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.sentiment import SentimentIntensityAnalyzer
import json
import pickle

words=[]
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']
intents_file = open("/content/drive/MyDrive/RPPOOPMiniProject/intents.json").read()
intents = json.loads(intents_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        #add documents in the corpus
        documents.append((word, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
#print(documents)
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
#print (len(documents), "documents")
# classes = intents
#print (len(classes), "classes", classes)
# words = all words, vocabulary
#print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# create our training data
training = []
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for word in words:
        bag.append(1) if word in pattern_words else bag.append(0)
        
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
# Create the model
model = Sequential([
    Dense(128, input_shape=(len(train_x[0]),), activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(train_y[0]), activation='softmax')
])

# Compile the model
optimizer = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Fit the model
history = model.fit(np.array(train_x), np.array(train_y), epochs=300, batch_size=5, verbose=1)

# Save the model
model.save('WBot')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
<ipython-input-5-42bd4c96de47>:68: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


Training data created
Epoch 1/300
35/35 [==============================] - 2s 5ms/step - loss: 3.7092 - accuracy: 0.0405
Epoch 2/300
35/35 [==============================] - 0s 4ms/step - loss: 3.6345 - accuracy: 0.0983
Epoch 3/300
35/35 [==============================] - 0s 6ms/step - loss: 3.5521 - accuracy: 0.0925
Epoch 4/300
35/35 [==============================] - 0s 4ms/step - loss: 3.4545 - accuracy: 0.1156
Epoch 5/300
35/35 [==============================] - 0s 4ms/step - loss: 3.3764 - accuracy: 0.1387
Epoch 6/300
35/35 [==============================] - 0s 3ms/step - loss: 3.3022 - accuracy: 0.1040
Epoch 7/300
35/35 [==============================] - 0s 3ms/step - loss: 3.1658 - accuracy: 0.1329
Epoch 8/300
35/35 [==============================] - 0s 3ms/step - loss: 3.0425 - accuracy: 0.1734
Epoch 9/300
35/35 [==============================] - 0s 4ms/step - loss: 2.8807 - accuracy: 0.2543
Epoch 10/300
35/35 [==============================] - 0s 4ms/step - loss: 2.8102 - accu

In [6]:
from keras.models import load_model
model = load_model('WBot')
intents = json.loads(open("/content/drive/MyDrive/RPPOOPMiniProject/intents.json").read())
words = pickle.load(open("words.pkl",'rb'))
classes = pickle.load(open("classes.pkl",'rb'))
def clean_up_sentence(sentence):
    # tokenize the pattern - splitting words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stemming every word - reducing to base form
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for words that exist in sentence
def bag_of_words(sentence, words, show_details=True):
    # tokenizing patterns
    sentence_words = clean_up_sentence(sentence)
    # bag of words - vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,word in enumerate(words):
            if word == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                #if show_details:
                    #print ("found in bag: %s" % word)
    return(np.array(bag))
def predict_class(sentence):
    # filter below  threshold predictions
    p = bag_of_words(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sorting strength probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents: 
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def responsed(msg1):
   # msg.append(msg1)
    ints = predict_class(msg1)
    response = getResponse(ints, intents)
    return response

In [7]:
class chatbot:
    def __init__(self):
        self.name = "stark"
        self.msgs = []
        
    def conversation(self):
        print(f"Chatbot : Hey there, How are you doing today? I am {self.name}.\n")
        flag = True
        while(flag == True):
            m = input("User : ")
            m = m.lower()
            self.msgs.append(m)
            if len(self.msgs) > 15:
                self.msgs = self.msgs[-15:]
            if(m == "bye" ):
                flag = False
                response = responsed(m)
                print("Chatbot : "+response)
            else:
                response = responsed(m)
                if(response =="Here are some songs for you" or response == "right here for you" ):
                    print("Chatbot : "+response)
                    response = self.song_emotion()
                print("Chatbot : "+response)
                
    def song_emotion(self):
        sid = SentimentIntensityAnalyzer()
        songs = {}

        for msg in self.msgs:
            sentiment_scores = sid.polarity_scores(msg)
            emotion = max(sentiment_scores, key=sentiment_scores.get)
            if(emotion == "neg"):
                emotion = "sad"
           # print(emotion)

            url = f"http://ws.audioscrobbler.com/2.0/?method=tag.gettoptracks&tag={emotion}&api_key=715156ee90d656693aaf300097923cbd&format=json&limit=20"
            response = requests.get(url)
            payload = response.json()

            for i in range(10):
                r = payload['tracks']['track'][i]
                songs[r['name']] = r['url']

        song_urls = list(songs.values())

        if len(song_urls) > 5:
            random.shuffle(song_urls)
            song_urls = song_urls[:5]

        return '\n'.join(song_urls)

In [ ]:
c = chatbot()
c.name
c.conversation()

Chatbot : Hey there, How are you doing today? I am stark.

User : Avani
1/1 [==============================] - 0s 88ms/step
Chatbot : Alright
User : suggest songs
1/1 [==============================] - 0s 18ms/step
Chatbot : right here for you
Chatbot : https://www.last.fm/music/Omi/_/Cheerleader
https://www.last.fm/music/Avalanche+City/_/Love+Love+Love
https://www.last.fm/music/Neu%21/_/Sonderangebot
https://www.last.fm/music/Geographer/_/Verona
https://www.last.fm/music/Portugal.+The+Man/_/Work+All+Day
User : sad
1/1 [==============================] - 0s 21ms/step
Chatbot : Awe thats unfortunate.
User : joke
1/1 [==============================] - 0s 29ms/step
Chatbot : What do you get when you cross a dyslexic, an insomniac, and an agnostic?
Someone who lays awake at night wondering if there is a dog.
User : suggest sad songs
1/1 [==============================] - 0s 22ms/step
Chatbot : Here are some songs for you
Chatbot : https://www.last.fm/music/Neu%21/_/Sonderangebot
https://www

KeyboardInterrupt: ignored

In [8]:
import json
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from twilio.rest import Client
from flask import Flask, request, jsonify

app = Flask(__name__)

twilio_phone_number = '+1(415)5238886'
user_phone_number = '+918259886081'

msg = list()

@app.route('/')
def hello():
  return "hello"

@app.route('/responsed', methods=['POST'])
def responsed():
    msg1 = request.form.get('msg1', '')
    msg.append(msg1)
    sentiment = analyze_sentiment(msg1)
    res = get_response(sentiment)
    return jsonify({'response': res})

@app.route('/song_emotion', methods=['POST'])
def song_emotion():
    sentiment = analyze_sentiment(' '.join(msg[-5:]))
    emotion = sentiment['emotion']
    songs = get_song_recommendations(emotion)
    return jsonify({'emotion': emotion, 'songs': songs})

def analyze_sentiment(text):
    sid = SentimentIntensityAnalyzer()
    sentiment_scores = sid.polarity_scores(text)
    sentiment = sentiment_scores['compound']
    return {'sentiment': sentiment}

def get_response(sentiment):
    # Modify this function to generate a response based on sentiment
    # For simplicity, let's assume a positive sentiment generates a positive response
    if sentiment['sentiment'] >= 0:
        return "That's great!"
    else:
        return "I'm sorry to hear that. Can I help you with something?"

def get_song_recommendations(emotion):
    # Modify this function to fetch song recommendations based on the emotion
    # Return a list of song names or URLs
    return ['Song 1', 'Song 2', 'Song 3']

@app.route('/sms', methods=['POST'])
def sms_endpoint():
    msg = request.form.get('Body', '')
    res = responsed(msg)
    send_sms(res['response'])
    return 'OK'

def send_sms(response):
    twilio_client.messages.create(
        body=response,
        from_=twilio_phone_number,
        to=user_phone_number
    )

if __name__ == '__main__':
    nltk.download('vader_lexicon')
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [9]:
from flask import Flask, request
from twilio.twiml.messaging_response import MessagingResponse

app = Flask(__name__)

@app.route('/sms', methods=['POST'])
def sms():
    message_body = request.form['Body']
    response = responsed(message_body)  # Get the chatbot response
    send_message(response)  # Send the response via Twilio
    return str(response)

def send_message(message):
    client.messages.create(
        body=message
    )


if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
